# Generate one year of demand based on calendar

In [1]:
import holidays
import yaml

from copy import deepcopy
from datetime import date, timedelta

In [2]:
with open('../configs/renfe_tft/demand_data_base.yaml', 'r') as f:
    demand_yaml = f.read()

data = yaml.safe_load(demand_yaml)

In [3]:
class Calendar1(holidays.HolidayBase):

    def _populate(self, year):
        # Febraury
        for i in range(1, 29):
            self[date(year, 2, i)] = 'Holidays'
        # March - October
        for i in range(1, 32):
            self[date(year, 3, i)] = 'Holidays'
            self[date(year, 10, i)] = 'Holidays'
        # June - September - November
        for i in range(1, 31):
            self[date(year, 6, i)] = 'Holidays'
            self[date(year, 9, i)] = 'Holidays'
            self[date(year, 11, i)] = 'Holidays'

In [4]:
class Calendar2(holidays.HolidayBase):

    def _populate(self, year):
        # January - May - July - August - December
        for i in range(1, 32):
            self[date(year, 1, i)] = 'Holidays'
            self[date(year, 5, i)] = 'Holidays'
            self[date(year, 7, i)] = 'Holidays'
            self[date(year, 8, i)] = 'Holidays'
            self[date(year, 12, i)] = 'Holidays'
        # April
        for i in range(1, 31):
            self[date(year, 4, i)] = 'Holidays'

In [5]:
class CalendarMadrid(holidays.HolidayBase):
    
    def _populate(self, year):
        self[date(year, 3, 20)] = 'Holidays'
        self[date(year, 5, 2)] = 'Holidays'
        self[date(year, 5, 15)] = 'Holidays'
        self[date(year, 11, 9)] = 'Holidays'

In [6]:
class CalendarBCN(holidays.HolidayBase):
    
    def _populate(self, year):
        self[date(year, 4, 10)] = 'Holidays'
        self[date(year, 5, 29)] = 'Holidays'
        self[date(year, 6, 24)] = 'Holidays'
        self[date(year, 9, 25)] = 'Holidays'

In [7]:
class CalendarOrange(holidays.HolidayBase):

    def _populate(self, year):
        self[date(year, 5, 27)] = 'Orange Warning'
        self[date(year, 5, 28)] = 'Orange Warning'
        self[date(year, 9, 15)] = 'Orange Warning'
        self[date(year, 9, 16)] = 'Orange Warning'

In [8]:
class CustomHolidays(holidays.ES):
    
    def _populate(self, year):
        holidays.ES._populate(self, year)
        for i in range(1, 7):
            self[date(year, 1, i)] = 'Navidad'
        self[date(year, 2, 14)] = 'San Valentín'
        for i in range(8, 14):
            self[date(year, 3, i)] = 'Carnaval'
        for i in range(2, 10):
            self[date(year, 4, i)] = 'Semana Santa'
        self[date(year, 8, 1)] = 'Verano'
        self[date(year, 8, 31)] = 'Verano'
        for i in range(24, 26):
            self[date(year, 12, i)] = 'Navidad'

In [9]:
es_holidays = holidays.ES()
madrid_holidays = CalendarMadrid()
BCN_holidays = CalendarBCN()
calendar1 = Calendar1()
calendar2 = Calendar2()
calendar_orange = CalendarOrange()

def get_demand_pattern(_date: date):
    demand_pattern = None
    if _date.weekday() >= 0 and _date.weekday() <= 3:
        if _date in calendar1:
            demand_pattern = 1 # Monday-Thrusday-1
        elif _date in calendar2:
            demand_pattern = 3 # Monday-Thrusday-2
    elif _date.weekday() > 3 and _date.weekday() <= 6:
        if _date in calendar1:
            demand_pattern = 2 # Weekend-1
        elif _date in calendar2:
            demand_pattern = 4 # Weekend-2
    if _date in es_holidays:
        demand_pattern = 4 # Weekend-2
    if _date in madrid_holidays or _date in BCN_holidays:
        demand_pattern = 5 # Local-Regional-Holidays
    if _date in calendar_orange:
        demand_pattern = 6 # Orange-Warning
    return demand_pattern

In [10]:
start_date = date(2023, 1, 1)
end_date = date(2023, 12, 31)
while start_date < end_date:
    previous_day = deepcopy(data['day'][-1])
    previous_day['id'] += 1
    previous_day['date'] += timedelta(days=1)
    previous_day['demandPattern'] = get_demand_pattern(previous_day['date'])
    data['day'].append(previous_day)
    start_date += timedelta(days=1)

In [11]:
yaml.safe_dump(data, open('../configs/renfe_tft/demand_data.yaml', 'w'), sort_keys=False, allow_unicode=True)